In [1]:
import pandas as pd
df = pd.read_csv("data/train_preliminary/lstm_5.csv")

In [2]:
df.shape

(27000000, 23)

In [3]:
df.head()

,Unnamed: 0,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,0,20.0,1.0,-0.044800,-0.019390,-0.031851,0.030554,-0.040778,-0.001448,-0.009343,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,0.030851,-0.023410,0.020876,0.027672,0.029498
1,1,20.0,1.0,0.043076,-0.025291,-0.006318,0.018357,-0.021721,0.033837,-0.038700,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.033420,-0.042072,0.049573,0.038932,-0.041535
2,2,20.0,1.0,-0.043862,-0.046132,0.047778,-0.008464,0.017775,0.039120,-0.046248,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.019325,-0.047091,0.022594,0.047237,-0.004500
3,3,39.0,1.0,0.037853,0.040882,-0.035817,-0.036202,0.043400,0.005398,-0.004463,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.025302,0.041088,-0.049014,0.032754,-0.013170
4,4,40.0,1.0,-0.044382,0.048629,-0.025343,-0.041154,-0.023619,0.018380,-0.037460,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.015037,-0.022605,0.024558,-0.040403,-0.015491


In [5]:
df = df.iloc[:, 1:]
df.head()

,time,click_times,creative_id_0,creative_id_1,creative_id_2,creative_id_3,creative_id_4,ad_id_0,ad_id_1,ad_id_2,...,product_category_0,product_category_1,product_category_2,product_category_3,product_category_4,advertiser_id_0,advertiser_id_1,advertiser_id_2,advertiser_id_3,advertiser_id_4
0,20.0,1.0,-0.044800,-0.019390,-0.031851,0.030554,-0.040778,-0.001448,-0.009343,0.004232,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,0.030851,-0.023410,0.020876,0.027672,0.029498
1,20.0,1.0,0.043076,-0.025291,-0.006318,0.018357,-0.021721,0.033837,-0.038700,0.039421,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.033420,-0.042072,0.049573,0.038932,-0.041535
2,20.0,1.0,-0.043862,-0.046132,0.047778,-0.008464,0.017775,0.039120,-0.046248,-0.046503,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.019325,-0.047091,0.022594,0.047237,-0.004500
3,39.0,1.0,0.037853,0.040882,-0.035817,-0.036202,0.043400,0.005398,-0.004463,-0.022102,...,-0.015297,0.001715,0.032569,0.027103,-0.003534,-0.025302,0.041088,-0.049014,0.032754,-0.013170
4,40.0,1.0,-0.044382,0.048629,-0.025343,-0.041154,-0.023619,0.018380,-0.037460,0.043368,...,0.017035,0.035791,-0.014875,-0.024798,-0.026263,-0.015037,-0.022605,0.024558,-0.040403,-0.015491


In [7]:
import numpy as np
X = np.array(df).reshape((-1, 30, 22))

In [8]:
df = None
df1 = pd.read_csv("data/train_preliminary/user.csv")

In [9]:
df1.head()

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1


In [22]:
Y = np.array(df1['age'])

In [23]:
Y = pd.get_dummies(Y)

In [24]:
Y = np.array(Y)

In [27]:
Y.shape

(900000, 10)

In [30]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.core import Masking
model = []
for i in range(10):
    X_train, X_test = X[:800000, :, :], X[800000:, :, :]
    Y_train, Y_test = Y[:800000, i], Y[800000:, i]
    model.append(Sequential())
    model[i].add(Masking(mask_value=0., input_shape=(30, 22)))
    model[i].add(LSTM(32))
    model[i].add(Dense(1))
    model[i].compile(loss='MSE', optimizer='adam')
    model[i].fit(X_train, Y_train, epochs=2, use_multiprocessing=True, workers=4, shuffle=True)
    Y_predict = model[i].predict(X_train)
    Y_predict = np.round(Y_predict)
    ans = 0
    for j in range(800000):
        if Y_predict[j] == Y_train[j]:
            ans += 1
    print('age', i + 1, 'accurency:', ans/800000)

Epoch 1/2
800000/800000 [==============================] - 496s 621us/step - loss: 0.0375
Epoch 2/2
800000/800000 [==============================] - 509s 636us/step - loss: 0.0373
age 800000 accurency: 0.96098125
Epoch 1/2
800000/800000 [==============================] - 536s 670us/step - loss: 0.1384
Epoch 2/2
800000/800000 [==============================] - 535s 668us/step - loss: 0.1374
age 800000 accurency: 0.83424875
Epoch 1/2
800000/800000 [==============================] - 522s 652us/step - loss: 0.1750
Epoch 2/2
800000/800000 [==============================] - 518s 647us/step - loss: 0.1744
age 800000 accurency: 0.77447
Epoch 1/2
800000/800000 [==============================] - 522s 652us/step - loss: 0.1399
Epoch 2/2
800000/800000 [==============================] - 510s 637us/step - loss: 0.1393
age 800000 accurency: 0.8327
Epoch 1/2
800000/800000 [==============================] - 496s 620us/step - loss: 0.1246
Epoch 2/2
800000/800000 [==============================] - 501s 6

In [32]:
import xgboost as xgb
Y_train, Y_test = np.array(df1['age'])[:800000], np.array(df1['age'])[800000:]
X_ready = model[0].predict(X)
for i in range(1, 10, 1):
    X_tmp = model[i].predict(X)
    X_ready = np.hstack((X_ready, X_tmp))
X_train, X_test = X_ready[:800000, :], X_ready[800000:, :]
modelx = xgb.XGBClassifier()
modelx.fit(X_train, Y_train)
modelx.score(X_train, Y_train)

0.25655625

In [33]:
modelx.score(X_test, Y_test)

0.22613